In [1]:
# Install the plaidml backend
import plaidml.keras
plaidml.keras.install_backend()

In [28]:
import pandas as pd
import numpy as np
import os
import sys
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import tqdm

In [16]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import cv2

import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, Activation, Dropout, GlobalAveragePooling2D, \
    BatchNormalization, concatenate, AveragePooling2D
from keras.optimizers import Adam

# 1. Input data

In [10]:
PATH_DATA = "/Users/david/TFM_DATA/spec"
data_files = os.listdir(PATH_DATA)

In [11]:
# PATH triplets
PATH_TRIPLETS = os.path.join("..","triplets", "triplets.csv")
df = pd.read_csv(PATH_TRIPLETS, delimiter= ";")

In [13]:
# List
triplets_input = list(df["output"])
size_triplets_input = len(triplets_input)

# Sample
triplets_input_v1 = triplets_input[:1000]
size_triplets_input = len(triplets_input_v1)

# Artists
artists_labels = np.sort(df.a1.unique())
df_artists = pd.DataFrame(data = artists_labels, columns = ["artist"]).reset_index()
df_artists.columns = ["id","artist"]

# DF Artists with artists as index
df_artists_index = df_artists.set_index("artist")
num_artists = df_artists.shape[0]

## Labels 

In [150]:
labels_mat = keras.utils.to_categorical(df_artists_index.id)

In [156]:
labels = dict()
for i, row in df_artists.iterrows():
    labels[row.artist] = labels_mat[row.id,].astype(int)

## 1.1 Dataframe Classifier

In [117]:
df1 = df[["a1","tr1","win1","ini1","fin1"]].copy()
df2 = df[["a1","tr2","win2","ini2","fin2"]].copy()

# Column names
colnames = ["art","tr","win","ini","fin"]
df1.columns = colnames
df2.columns = colnames

In [120]:
df_concat = pd.concat([df1, df2])

In [127]:
df_concat["tr"] = df_concat.tr + "__" + df_concat.win.astype(str) +  \
    "__" + df_concat.ini.astype(str) + "__" + df_concat.fin.astype(str) + ".jpg"

## 1.2 Data Generator

In [206]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, df, x_col, y_col=None, batch_size=32, num_classes=None, shuffle=True):
        self.batch_size = batch_size
        self.df = df
        self.indices = self.df.index.tolist()
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.x_col = x_col
        self.y_col = y_col
        self.dim = (x_col, y_col, 1) #one input channel
        self.on_epoch_end()

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, index):
        idx = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in idx]
        
        X, y = self.__get_data(batch)
        return X, y

    def on_epoch_end(self):
        pass

    def __get_data(self, batch):
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size, self.num_classes), dtype=int)
        
        # Get the list of image files and corresponding artists
        df_imgs_files = self.df.iloc[batch]
        
        list_imgs = list(df_imgs_files["tr"])
        list_art = list(df_imgs_files["art"])
        
        for ii in range(len(list_imgs)):
            
            # Read image using cv2
            path_img = os.path.join(PATH_DATA, list_imgs[ii])
            img = cv2.cvtColor(cv2.imread(path_img), cv2.COLOR_BGR2GRAY)
            img = np.round(img / 255.,5)
            img = cv2.resize(img, (self.y_col, self.x_col))
            img = np.expand_dims(img, axis = 2) # add the dimension of the channel 
            
            # Put it into X matrix
            X[ii,] = img
            y[ii] = labels[list_art[ii]]

        return X, y

In [207]:
x_col, y_col = (256, 937)
batch_size = 10
num_classes = df_artists.shape[0]
shuffle = False

training_generator = DataGenerator(df=df_concat, x_col=x_col, y_col=y_col,
                                   batch_size=batch_size, num_classes=num_artists,
                                   shuffle=False)

# Model